In [1]:
import pandas as pd 
import urllib3
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import os
from datetime import datetime
from tqdm import tqdm
from datetime import date
import itertools

import warnings
warnings = 1

In [2]:
def convert_date(d):
    try:
        date = datetime.strptime(d, '%Y %b %d').strftime('%Y-%m-%d')
    except:
        date = datetime.strptime(d, '%Y %b').strftime('%Y-%m')
    return date

In [3]:
def clean_carriages(array):
    try:
        array = [line.replace('\r', '') for line in array]
    except:
        print('no return carriages')
    
    try:
        array = [line.replace('\n', '') for line in array]
    except:
        print('no newline carriages')
    
    return(array)

# might be able to use soup.stripped_strings:

In [4]:
def get_PI(grantDict, grantNumber):
    a = []
    for grants in grantNumber:
        a.append(
            grantDict.loc[grantDict['Grant'] == grants].reset_index().PI[0].split(",")[0]
        )

    return(a)

    

In [5]:
def get_df_info(pubmed_url, ids, grant_names_1):
    url = pubmed_url + ids
    
    
    page = urlopen(url)
    html_doc = page.read()
    html = html_doc.decode("utf-8")

    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    # =============================================== #
    # preprint or not:
    if (len(re.findall('preprint\s', str(soup))) < 1):
        print_type = ''
    else:
        print_type = 'preprint'
    
    # =============================================== #
    # Citation (since im lazy)
    try:
        citation = soup.find("div", {"class": "article-citation"}).text.replace('\r', '').replace('\n', '')
        citation = re.findall("(^.*?)(?:doi)", citation)[0].strip()
    except:
        citation = ''
    
    # =============================================== #
    # Finding by class name (title)
    title = soup.find(class_ = "heading-title")
    try:
        title = str(clean_carriages(title)[0]).strip()
    except:
        title = str(title)
        title = title.replace('\n', '').replace('<h1 class="heading-title">','').replace('</h1>','')
        try:
            title = title.replace('<i>', '').replace('</i>','').strip()

        except:
            print("no italics")
    
    # =============================================== #
    # Finding by class name (authors)
    author = soup.find_all(class_ = "authors-list-item")
    author_name = re.findall("data-ga-label=\"(.+?)\"", str(author))
    
    # =============================================== #
    # Finding date of publication (cit)
#     DOP = soup.find(class_ = "cit").string
#     try:
#         date = convert_date(DOP.split(';')[0])
#     except:
#         date = convert_date(DOP.split('.')[0])
    
    # =============================================== #
    # Finding the correct grant
    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    grant = soup.find_all(class_ = "grant-item")
    grant = re.findall("data-ga-label=\"(.+?)\"", str(grant))

    str_grants = ', '.join(grant)
    # print(str_grants)
    
    # =============================================== #
    # clean out grants that are not in the right format:
    # bgrants = re.findall('(\w{3} \d{2}X\d{3})|(CA\d{6})', ' '.join(grant))
    bgrants = re.findall('(\w{3} CA\d{6})|(\w{3} \d{2}X\d{3})|(CA\d{6}|(\d{2}\w\d{5}\w\d{5}))', ' '.join(grant))
    out = list(filter(None,itertools.chain(*bgrants)))
    
    for i in out:
        base_grant = ''
        seroNet_grant = ''

        try:
            grant = i.split(' ')[1]
        except:
            grant = i

#         print(grant)

        try:
            if grant in grant_names_1:
                base_grant = grant
                seroNet_grant = grant
                break
        except:
            print('error')
    
    # Finding publishing journal 
    soup = BeautifulSoup(html_doc, 'lxml', from_encoding='utf-8')
    journal = re.findall('(?:meta content=\")(.*?)(?:name=\"citation_journal_title)',str(soup))[0][:-2]
    
    
    return title, author_name, journal, seroNet_grant, base_grant, print_type, citation
    # # Finding by class name (title)
    # title = soup.find(class_ = "heading-title")
    # title = str(clean_carriages(title)[0]).strip()

In [6]:
#  page = urlopen('https://pubmed.ncbi.nlm.nih.gov/35055463/')
# html_doc = page.read()
# html = html_doc.decode("utf-8")

# soup = BeautifulSoup( html_doc , 'html.parser')

# re.findall('preprint\s', str(soup))

# grant = soup.find_all(class_ = "grant-item")
# grant = re.findall("data-ga-label=\"(.+?)\"", str(grant))

# str_grants = ', '.join(grant)
# bgrants = re.findall('(\w{3} CA\d{6})|(\w{3} \d{2}X\d{3})|(CA\d{6}|(\d{2}\w\d{5}\w\d{5}))', ' '.join(grant))
# out = list(filter(None,itertools.chain(*bgrants)))
# print(out)
# for i in out:
#     base_grant = ''
#     seroNet_grant = ''

#     try:
#         grant = i.split(' ')[1]
#     except:
#         grant = i

# #         print(grant)

#     try:
#         if grant in grant_names:
#             base_grant = grant
#             seroNet_grant = grant
#             break
#     except:
#         print('error')

## Pull Papers ##

In [10]:
PUBMED_SEARCH = 'https://pubmed.ncbi.nlm.nih.gov/?term=CA260508+OR+CA260476+OR+CA260539+OR+CA' \
'60584+OR+CA260507+OR+CA260469+OR+CA260526+OR+CA260588+OR+CA260513+OR+CA260541+OR+CA260462'\
'+OR+CA261276+OR+CA261277+OR+CA260582+OR+CA260543+OR+CA260492+OR+CA260591+OR+CA260581+OR+' \
'CA260517+OR+CA260563+OR+CA260560+OR+21X089+OR+21X090+OR+21X091+OR+21X092%5BGrant+Number%5'\
'D&format=pubmed&sort=date&size=200'

BASE_DIR = '~/National Institutes of Health/NCI-FNL SeroNet Team - Curation channel/'
FILE_NAME = 'Award to curator and all contacts mapping.xlsx'
grant_map = pd.read_excel(os.path.join(BASE_DIR, FILE_NAME))

# grant_names
grant_names = re.findall('(CA\d{6})', PUBMED_SEARCH)
grant_names += re.findall('(\d{2}X\d{3})', PUBMED_SEARCH)
grant_names.append('75N91019D00024')

today = date.today()
today = today.strftime("%y-%m-%d")

In [11]:
# Input
PMID_input = '34794169 35075455 35090580 35090596'

PMID_list = PMID_input.split(' ')
search_url = 'https://pubmed.ncbi.nlm.nih.gov/'

In [12]:
# Output
OUT_DIR = f'./new_pmids/data_pull_{today}.csv'
JENN_OUT_DIR = f'./for_jenn/data_pull_{today}.csv'

In [13]:
PMID = []
title_list = []
author_list = []
journal_list = []
grant_list = []
base_list = []
paperType_list = []
paper_citation = []
curator = []

for ids in tqdm(PMID_list):
    title, author_name, journal, seroNet_grant, base_grant, print_type, citation = get_df_info(search_url, ids, grant_names)
    
    PMID.append(ids)
    title_list.append(title)
    author_list.append(str(author_name))
    journal_list.append(journal)
#     date_list.append(date_str)
    grant_list.append(seroNet_grant)
    base_list.append(base_grant)
    paperType_list.append(print_type)
    paper_citation.append(citation)
    
    
grantCuratorDic = dict(zip(grant_map['Grant'], grant_map['primaryCurator']))

curator = [*map(grantCuratorDic.get,grant_list)]

# replacing 75N91019D00024 with 75N91019D00024, Task Order No. 75N91020F00003
base_list = [sub.replace('75N91019D00024',
                         '75N91019D00024, Task Order No. 75N91020F00003') for sub in base_list]


100%|██████████| 4/4 [00:03<00:00,  1.33it/s]


In [14]:
empty = ['']*len(PMID)
OWNER = 'Publication Owner'
EPIC_NUMBER = 'JIRA Epic #'
BASE_PROJ = 'Base_Project'
STATUS = 'Status/ImmPort Study ID'
BASE_PROJ_1 = 'Base_Project.1'
NUMBER = 'PMID'
PMID_NUMBER = 'PMID_N'
URL = 'PMID_link'
TITLE = 'Publication_Title'
JOURNAL = 'Journal'
AUTHOR = 'Author(s)'
DATE = 'Pub Date'
SERONET_PI = 'PI'
PRE_PRINT = 'print_type'
CITE = 'citation'


updating_fields = {
    OWNER: curator,
    EPIC_NUMBER: empty,
    BASE_PROJ: base_list,
    STATUS: empty,
    BASE_PROJ_1: grant_list, 
    NUMBER: PMID,
    PMID_NUMBER: ['PMID_'+str(x) for x in PMID],
    URL: [search_url+str(x) for x in PMID],
    TITLE: title_list, 
    JOURNAL:journal_list,
    AUTHOR:author_list,
    DATE:empty,
    SERONET_PI:empty,
    PRE_PRINT: paperType_list,
    CITE: paper_citation
}

In [15]:
temp = pd.DataFrame(updating_fields)
# display(temp)
temp[AUTHOR] = temp[AUTHOR].str.replace('\'','')
temp[AUTHOR] = temp[AUTHOR].str.replace('[','')
temp[AUTHOR] = temp[AUTHOR].str.replace(']','')

temp[SERONET_PI] = get_PI(grant_map, temp[BASE_PROJ_1])


temp.to_csv(OUT_DIR)
temp[['PMID','Publication_Title']].to_csv(JENN_OUT_DIR)

temp.head()

<ipython-input-15-045c6c8f323a>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp[AUTHOR] = temp[AUTHOR].str.replace('[','')
<ipython-input-15-045c6c8f323a>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp[AUTHOR] = temp[AUTHOR].str.replace(']','')


,Publication Owner,JIRA Epic #,Base_Project,Status/ImmPort Study ID,Base_Project.1,PMID,PMID_N,PMID_link,Publication_Title,Journal,Author(s),Pub Date,PI,print_type,citation
0,Liu / Mani,,CA260476,,CA260476,34794169,PMID_34794169,https://pubmed.ncbi.nlm.nih.gov/34794169,Optimization of non-coding regions for a non-m...,Nature,"Makda S Gebre, Susanne Rauch, Nicole Roth, Jin...",,BAROUCH,,Nature. 2022 Jan;601(7893):410-414.
1,Liu,,CA260517,,CA260517,35075455,PMID_35075455,https://pubmed.ncbi.nlm.nih.gov/35075455,Autoantibodies targeting cytokines and connect...,Research square,"Allan Feng, Emily Yang, Andrew Moore, Shaurya ...",,BOYD,,Res Sq. 2022 Jan 20;rs.3.rs-1233038.
2,Liu / Mani,,CA260476,,CA260476,35090580,PMID_35090580,https://pubmed.ncbi.nlm.nih.gov/35090580,mRNA-1273 vaccine-induced antibodies maintain ...,Immunity,"Paulina Kaplonek, Stephanie Fischinger, Deniz ...",,BAROUCH,,Immunity. 2022 Jan 6;55(2):355-365.e4.
3,Mani,,CA260543,,CA260543,35090596,PMID_35090596,https://pubmed.ncbi.nlm.nih.gov/35090596,SARS-CoV-2 mRNA vaccine induces robust specifi...,Cell reports,"Tara M Narowski, Kristin Raphel, Lily E Adams,...",,BARIC,,Cell Rep. 2022 Feb 1;38(5):110336.


In [16]:
for i,o in enumerate(temp.PMID):
    if temp[PRE_PRINT][i] != 'preprint':
        print(f"CURATOR: {temp[OWNER][i]}")
        print(temp[PMID_NUMBER][i])
        print(f"*Title*: {temp[TITLE][i]}")
        print(f"*Journal*: {temp[JOURNAL][i]}")
        print(f"*Pubmed*: {temp[URL][i]}")
        print(f"CURATOR: {temp[OWNER][i]}")
        print(temp[PMID_NUMBER][i])
        print(temp[SERONET_PI][i].capitalize())
        print(temp[BASE_PROJ_1][i])
        print("\n")

print("\n\n")
print("These are the new publications for this week:")
for i,o in enumerate(temp.PMID):
    
    if temp[OWNER][i] == 'Mani' or temp[OWNER][i] == 'Liu / Mani':
        print(temp[PMID_NUMBER][i])

CURATOR: Liu / Mani
PMID_34794169
*Title*: Optimization of non-coding regions for a non-modified mRNA COVID-19 vaccine
*Journal*: Nature
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/34794169
CURATOR: Liu / Mani
PMID_34794169
Barouch
CA260476


CURATOR: Liu
PMID_35075455
*Title*: Autoantibodies targeting cytokines and connective tissue disease autoantigens are common in acute non-SARS-CoV-2 infections
*Journal*: Research square
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/35075455
CURATOR: Liu
PMID_35075455
Boyd
CA260517


CURATOR: Liu / Mani
PMID_35090580
*Title*: mRNA-1273 vaccine-induced antibodies maintain Fc effector functions across SARS-CoV-2 variants of concern
*Journal*: Immunity
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/35090580
CURATOR: Liu / Mani
PMID_35090580
Barouch
CA260476


CURATOR: Mani
PMID_35090596
*Title*: SARS-CoV-2 mRNA vaccine induces robust specific and cross-reactive IgG and unequal neutralizing antibodies in naive and previously infected people
*Journal*: Cell repor